Необходимые импорты

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import recall_score
import seaborn as sns

Загрузка данных

In [87]:
data = load_breast_cancer()
X = data.data
y = data.target
feature_names = data.feature_names
target_names = data.target_names

print(f"Размер датасета: {X.shape}")
print(f"Количество признаков: {len(feature_names)}")
print(f"Целевые классы: {target_names}")
print(f"Соотношение классов: {np.bincount(y)}")


Размер датасета: (569, 30)
Количество признаков: 30
Целевые классы: ['malignant' 'benign']
Соотношение классов: [212 357]


Разделение на тренировочную и тестовую выборку


In [88]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain size: {X_train.shape}, Test size: {X_test.shape}")



Train size: (455, 30), Test size: (114, 30)


In [2]:
from decision_tree import DecisionTree
from sklearn.tree import DecisionTreeClassifier

модели

In [ ]:
обучение

In [90]:
lr = DecisionTree.DecisionTreeClassifier()
lrsk = DecisionTreeClassifier()
lr.fit(X_train, y_train)
lrsk.fit(X_train, y_train)

DecisionTreeClassifier()

предсказания

Используем реколл, так как не хотим пропустить больных при обследовании


In [91]:
y_pred = lr.predict(X_test)
y_pred_proba = lr.predict_proba(X_test)

y_pred_sk = lrsk.predict(X_test)
y_pred_proba_sk = lrsk.predict_proba(X_test)

In [92]:
print("Реколл для больных моей реализации", recall_score(y_test, y_pred, pos_label=0))
print("Реколл для больных склерн реализации", recall_score(y_test, y_pred_sk, pos_label=0))


Реколл для больных моей реализации 0.9761904761904762
Реколл для больных склерн реализации 0.9285714285714286


Нормализуем данные

In [93]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

С помощью кросс валидации подбираем гиперпараметры


In [95]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Создание пайплайна для решающего дерева
# Стандартизация обычно менее критична для деревьев, но может помочь при смешанных типах признаков
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # опционально для деревьев
    ('tree', DecisionTreeClassifier(random_state=42))
])

param_grid = {
    'tree__criterion': ['gini', 'entropy'],
    'tree__max_depth': [5, 10, 15, None],
    'tree__min_samples_split': [2, 10, 20],
    'tree__min_samples_leaf': [1, 5, 10]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,  # или param_grid_fast для ускорения
    cv=5,
    scoring='recall',  # или 'accuracy', 'precision', 'f1', 'roc_auc'
    n_jobs=-1,
    verbose=1
)

print("\nНачинаем поиск оптимальных параметров для решающего дерева...")
grid_search.fit(X_train, y_train)

print(f"\nЛучшие параметры: {grid_search.best_params_}")
print(f"Лучший recall (CV): {grid_search.best_score_:.4f}")

# Получаем лучшую модель
best_tree_model = grid_search.best_estimator_

# Оценка на тестовых данных
from sklearn.metrics import recall_score, classification_report

y_pred = best_tree_model.predict(X_test)
test_recall = recall_score(y_test, y_pred)
print(f"\nRecall на тестовых данных: {test_recall:.4f}")
print("\nПолный отчет по классификации:")
print(classification_report(y_test, y_pred))


Начинаем поиск оптимальных параметров для решающего дерева...
Fitting 5 folds for each of 72 candidates, totalling 360 fits

Лучшие параметры: {'tree__criterion': 'gini', 'tree__max_depth': 5, 'tree__min_samples_leaf': 1, 'tree__min_samples_split': 10}
Лучший recall (CV): 0.9719

Recall на тестовых данных: 0.9167

Полный отчет по классификации:
              precision    recall  f1-score   support

           0       0.86      0.90      0.88        42
           1       0.94      0.92      0.93        72

    accuracy                           0.91       114
   macro avg       0.90      0.91      0.91       114
weighted avg       0.91      0.91      0.91       114



Тестируем на лучшей модели

In [98]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

print("\n" + "="*50)
print("ОЦЕНКА МОДЕЛИ sklearn НА ТЕСТОВОЙ ВЫБОРКЕ")
print("="*50)

recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.4f}")


cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)


ОЦЕНКА МОДЕЛИ sklearn НА ТЕСТОВОЙ ВЫБОРКЕ
Recall: 0.9167
Confusion Matrix:
[[38  4]
 [ 6 66]]


Используем те же параметры на своей модели

In [3]:
lr = DecisionTree.DecisionTreeClassifier(max_depth=5, criterion='gini')

In [101]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [102]:
print("\n" + "="*50)
print("ОЦЕНКА МОДЕЛИ sklearn НА ТЕСТОВОЙ ВЫБОРКЕ")
print("="*50)

recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.4f}")


cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)


ОЦЕНКА МОДЕЛИ sklearn НА ТЕСТОВОЙ ВЫБОРКЕ
Recall: 0.9028
Confusion Matrix:
[[41  1]
 [ 7 65]]


Загружаем данные

In [4]:
from sklearn import datasets
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target

Делим на выборки

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Обучаем бейзлайн

In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [9]:
dt = DecisionTreeRegressor()

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

In [10]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"mse = {mse}")
print(f"mae = {mae}")
print(f"r2 = {r2}")

mse = 5111.988764044944
mae = 55.359550561797754
r2 = 0.0351373447398482


In [11]:
from decision_tree import DecisionTree

In [12]:
dt = DecisionTree.DecisionTreeRegressor()

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

In [13]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"mse = {mse}")
print(f"mae = {mae}")
print(f"r2 = {r2}")

mse = 5362.505617977528
mae = 57.157303370786515
r2 = -0.012146475321124006


In [14]:
Работаем над данными

SyntaxError: invalid syntax (ipython-input-460497718.py, line 1)

In [15]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [ ]:
Подбираем параметры

In [16]:
from sklearn.tree import DecisionTreeRegressor

pipeline = Pipeline([
    ('scaler', StandardScaler()),  # для деревьев можно убрать, но оставляем для консистентности
    ('tree', DecisionTreeRegressor(random_state=42))
])

# Упрощенная сетка параметров для быстрого поиска
param_grid = {
    'tree__max_depth': [5, 10, 15, None],  # самый важный параметр
    'tree__min_samples_leaf': [1, 5, 10],  # второй по важности
    'tree__criterion': ['squared_error', 'absolute_error']  # основные критерии
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,  # уменьшаем для скорости
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

print("\nНачинаем быстрый поиск оптимальных параметров для решающего дерева...")
grid_search.fit(X_train, y_train)

print(f"\nЛучшие параметры: {grid_search.best_params_}")
print(f"Лучший отрицательный MSE (CV): {grid_search.best_score_:.4f}")
print(f"Лучший MSE (CV): {-grid_search.best_score_:.4f}")

best_model = grid_search.best_estimator_


Начинаем быстрый поиск оптимальных параметров для решающего дерева...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

Лучшие параметры: {'tree__criterion': 'squared_error', 'tree__max_depth': 10, 'tree__min_samples_leaf': 10}
Лучший отрицательный MSE (CV): -4293.5981
Лучший MSE (CV): 4293.5981


Предсказываем на лучшей модели

In [17]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

In [18]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"mse = {mse}")
print(f"mae = {mae}")
print(f"r2 = {r2}")

mse = 3545.020730041851
mae = 47.286087113809565
r2 = 0.33089482930827707


Видим улучшение метрик


In [22]:
model = LinearRegression.LRAnalytical(reg='l2')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [24]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"mse = {mse}")
print(f"mae = {mae}")
print(f"r2 = {r2}")

mse = 3549.408310041851
mae = 50.043587113809565
r2 = 0.3184308293082771


Аналогично